In [64]:
import json 
import pandas as pd
import numpy as np
import uuid
import re
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD, DCTERMS, PROV, SKOS, RDFS
from functions import *
from namespaces import *
from shapely.geometry import shape
from datetime import datetime

In [65]:
#import json files
with open('/workspaces/ontologie-peuplement/data/gentilly/parcelles_1811.geojson') as f:
    data_1811 = json.load(f)

with open('/workspaces/ontologie-peuplement/data/gentilly/parcelles_1848.geojson') as f:
    data_1845 = json.load(f)
COMMUNE = 'Gentilly'

In [66]:
data_1845

{'type': 'FeatureCollection',
 'name': 'parcelles_1848',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::2154'}},
 'features': [{'type': 'Feature',
   'properties': {'fid': 1, 'numero': '1', 'section': 'D', 'nature': 'NB'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[652493.0942947898, 6859103.821806228],
      [652518.1291845728, 6859114.864593228],
      [652528.4860841817, 6859091.270066966],
      [652498.1698614856, 6859078.238206532],
      [652498.1012727465, 6859082.353530879],
      [652493.6430047029, 6859080.981756097],
      [652493.0942947898, 6859103.821806228]]]}},
  {'type': 'Feature',
   'properties': {'fid': 2, 'numero': '2', 'section': 'D', 'nature': 'NB-B'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[652518.1291845728, 6859114.864593228],
      [652532.7385860079, 6859121.174757228],
      [652543.1640743559, 6859097.785997184],
      [652528.4860841817, 6859091.270066966],
      [652518.1291845728, 6859114.864593228

In [67]:
COMMUNE = 'Gentilly'
matrices_metada = {
    "MAT_1813": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_B_NB_1813",
        "MATRICE_START": "1813",
        "MATRICE_END": "1835"
    },
    "MAT_1836": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_NB_1836",
        "MATRICE_START": "1836",
        "MATRICE_END": "1847"
    },
    "MAT_1848": {
        "PLAN": "1845",
        "MATRICE_ID": "MAT_NB_1848",
        "MATRICE_START": "1848",
        "MATRICE_END": "1860"
    }
}

In [68]:
#Créer un objet Turtle avec une URI
g = Graph()

g.bind('landmark', landmarkuri)
g.bind('owner', owneruri)
g.bind('source', srcuri)
g.bind('cad_ltype', cad_ltype)
g.bind('cad_atype', cad_atype)
g.bind('pnature',pnature)
g.bind('ctype',ctype)
g.bind('atype',atype)

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

In [69]:
def get_base_Skos_Concept(str):
    if str == 'B':
        return 'ProprieteBatie'
    elif str == 'NB':
        return 'ProprieteNonBatie'
    elif str == 'B-NB' or str == 'NB-B':
        return 'ProprieteBatie;ProprieteNonBatie'
    
def geojson_to_wkt(geojson_geometry):
    shapely_geometry = shape(geojson_geometry)
    return shapely_geometry.wkt

In [70]:
geojsons = [data_1811, data_1845]
for i in range(len(geojsons)):
    if i == 0:
        BEGIN = '1811-01-01'
        END = '1847-12-31'
        LANDMARK_SECTION = "87d7c2f6-306b-45a1-a833-5e17821c3102"
    elif i == 1:
        BEGIN = '1845-01-01'
        END = '1859-12-31'
        LANDMARK_SECTION = 'da6a5c2c-e86d-43bb-8950-7169bd0df60a'

    for elem in geojsons[i]['features']:
        resource_uri = URIRef(landmarkuri + str(uuid.uuid4()))
        g.add((resource_uri, RDF.type, add.Landmark))
        g.add((resource_uri, add.isLandmarkType, cad_ltype.Plot))
        g.add((resource_uri, DCTERMS.identifier, Literal(elem['properties']['section'] + '-' + elem['properties']['numero'], datatype=XSD.string)))
        g.add((resource_uri, SKOS.prefLabel, Literal(elem['properties']['section'] + '-' + str(elem['properties']['numero']) + ' ('+ COMMUNE +')', datatype=XSD.string)))
        g.add((resource_uri, RDFS.label, Literal(elem['properties']['section'] + '-' + str(elem['properties']['numero']) + ' ('+ COMMUNE +')', datatype=XSD.string)))

        #Plot Base Landmark Relation : fix Landmark Relation of the plot : Plot => Section => Commune
        landmarkRelation = BNode()
        g.add((URIRef(landmarkRelation.n3()), RDF.type, add.LandmarkRelation))
        g.add((URIRef(landmarkRelation.n3()), add.locatum, resource_uri))
        g.add((URIRef(landmarkRelation.n3()), add.relatum, URIRef(landmarkuri + LANDMARK_SECTION)))

        #Landmark Appearance
        ## Change
        changeapp = BNode()
        g.add((URIRef(changeapp.n3()), RDF.type, add.Change))
        g.add((URIRef(changeapp.n3()),add.appliedTo,resource_uri))
        g.add((URIRef(changeapp.n3()),add.isChangeType,ctype.LandmarkAppearance))
        
        ## Event
        eventapp = BNode()
        g.add((URIRef(eventapp.n3()), RDF.type, add.Event))
        g.add((URIRef(changeapp.n3()),add.dependsOn,URIRef(eventapp.n3())))
        time_ = BNode()
        g.add((URIRef(eventapp.n3()), add.hasTime, URIRef(time_.n3())))
        g.add((URIRef(time_.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(time_.n3()), add.timeCalendar,time.Gregorian))
        g.add((URIRef(time_.n3()), add.timePrecision,time.Year))
        g.add((URIRef(time_.n3()), add.timeValue, Literal(datetime.strptime(BEGIN, '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))

        #Landmark Disappearance
        ## Change
        changedis = BNode()
        g.add((URIRef(changedis.n3()), RDF.type, add.Change))
        g.add((URIRef(changedis.n3()),add.appliedTo,resource_uri))
        g.add((URIRef(changedis.n3()),add.isChangeType,ctype.LandmarkDisappearance))
        
        ## Event
        eventdis= BNode()
        g.add((URIRef(eventdis.n3()), RDF.type, add.Event))
        g.add((URIRef(changedis.n3()),add.dependsOn,URIRef(eventdis.n3())))
        time_ = BNode()
        g.add((URIRef(eventdis.n3()), add.hasLatestTimeInstant, URIRef(time_.n3())))
        g.add((URIRef(time_.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(time_.n3()), add.timeCalendar,time.Gregorian))
        g.add((URIRef(time_.n3()), add.timePrecision,time.Year))
        g.add((URIRef(time_.n3()), add.timeValue, Literal(datetime.strptime(END, '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))

        # Plot Nature
        ## Version
        natureattr = BNode()
        g.add((URIRef(natureattr.n3()),RDF.type,add.Attribute))
        g.add((resource_uri, add.hasAttribute, URIRef(natureattr.n3())))
        g.add((URIRef(natureattr.n3()), add.isAttributeType, cad_atype.PlotNature))
        natureattrversion = BNode()
        g.add((URIRef(natureattr.n3()), add.hasAttributeVersion,URIRef(natureattrversion.n3())))
        g.add((URIRef(natureattrversion.n3()),RDF.type,add.AttributeVersion))
        nature = get_base_Skos_Concept(elem['properties']['nature'])
        if ';' in nature:
            for n in nature.split(';'):
                g.add((URIRef(natureattrversion.n3()),add.hasPlotNature,URIRef(pnature+n)))
        else:
            g.add((URIRef(natureattrversion.n3()),add.hasPlotNature,URIRef(pnature+nature)))
        
        ## Plot Nature Appearance
        ### Change
        changenatapp = BNode()
        g.add((URIRef(changenatapp.n3()), RDF.type, add.Change))
        g.add((URIRef(changenatapp.n3()),add.appliedTo,URIRef(natureattr.n3())))
        g.add((URIRef(changenatapp.n3()),add.makesEffective,URIRef(natureattrversion.n3())))
        ### Event
        eventnatapp = BNode()
        g.add((URIRef(eventnatapp.n3()), RDF.type, add.Event))
        g.add((URIRef(changenatapp.n3()),add.dependsOn,URIRef(eventnatapp.n3())))
        time_ = BNode()
        g.add((URIRef(eventnatapp.n3()), add.hasTime, URIRef(time_.n3())))
        g.add((URIRef(time_.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(time_.n3()), add.timeCalendar,time.Gregorian))
        g.add((URIRef(time_.n3()), add.timePrecision,time.Year))
        g.add((URIRef(time_.n3()), add.timeValue, Literal(datetime.strptime(BEGIN, '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))

        ## Plot Nature Disappearance
        ### Change
        changenatdis = BNode()
        g.add((URIRef(changenatdis.n3()), RDF.type, add.Change))
        g.add((URIRef(changenatdis.n3()),add.appliedTo,URIRef(natureattr.n3())))
        g.add((URIRef(changenatdis.n3()),add.outdates,URIRef(natureattrversion.n3())))
        ### Event
        eventnatdis = BNode()
        g.add((URIRef(eventnatdis.n3()), RDF.type, add.Event))
        g.add((URIRef(changenatdis.n3()),add.dependsOn,URIRef(eventnatdis.n3())))
        time_ = BNode()
        g.add((URIRef(eventnatdis.n3()), add.hasLatestTimeInstant, URIRef(time_.n3())))
        g.add((URIRef(time_.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(time_.n3()), add.timeCalendar,time.Gregorian))
        g.add((URIRef(time_.n3()), add.timePrecision,time.Year))
        g.add((URIRef(time_.n3()), add.timeValue, Literal(datetime.strptime(END, '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))

        #Plot Geometry
        geom = BNode()
        g.add((URIRef(geom.n3()), RDF.type, add.Attribute))
        g.add((URIRef(geom.n3()), add.isAttributeType, atype.Geometry))
        g.add((resource_uri, add.hasAttribute, URIRef(geom.n3())))
        geomversion = BNode()
        g.add((URIRef(geom.n3()), add.hasAttributeVersion, URIRef(geomversion.n3())))
        g.add((URIRef(geomversion.n3()), RDF.type, add.AttributeVersion))
        g.add((URIRef(geomversion.n3()), add.versionValue, Literal(geojson_to_wkt(elem['geometry']) + "<http://www.opengis.net/def/crs/EPSG/0/2154>", datatype=geo.wktLiteral)))
        ## Plot Geometry Appearance
        ### Change
        changegeomapp = BNode()
        g.add((URIRef(changegeomapp.n3()), RDF.type, add.Change))
        g.add((URIRef(changegeomapp.n3()), add.appliedTo, URIRef(geom.n3())))
        g.add((URIRef(changegeomapp.n3()), add.makesEffective, URIRef(geomversion.n3())))
        ### Event
        eventgeomapp = BNode()
        g.add((URIRef(eventgeomapp.n3()), RDF.type, add.Event))
        g.add((URIRef(changegeomapp.n3()), add.dependsOn, URIRef(eventgeomapp.n3())))
        time_ = BNode()
        g.add((URIRef(eventgeomapp.n3()), add.hasTime, URIRef(time_.n3())))
        g.add((URIRef(time_.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(time_.n3()), add.timeCalendar, time.Gregorian))
        g.add((URIRef(time_.n3()), add.timePrecision, time.Year))
        g.add((URIRef(time_.n3()), add.timeValue, Literal(datetime.strptime(BEGIN, '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))

        ## Plot Geometry Disappearance
        ### Change
        changegeomdis = BNode()
        g.add((URIRef(changegeomdis.n3()), RDF.type, add.Change))
        g.add((URIRef(changegeomdis.n3()), add.appliedTo, URIRef(geom.n3())))
        g.add((URIRef(changegeomdis.n3()), add.outdates, URIRef(geomversion.n3())))
        ### Event
        eventgeomdis = BNode()
        g.add((URIRef(eventgeomdis.n3()), RDF.type, add.Event))
        g.add((URIRef(changegeomdis.n3()), add.dependsOn, URIRef(eventgeomdis.n3())))
        time_ = BNode()
        g.add((URIRef(eventgeomdis.n3()), add.hasLatestTimeInstant, URIRef(time_.n3())))
        g.add((URIRef(time_.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(time_.n3()), add.timeCalendar, time.Gregorian))
        g.add((URIRef(time_.n3()), add.timePrecision, time.Year))
        g.add((URIRef(time_.n3()), add.timeValue, Literal(datetime.strptime(END, '%Y-%m-%d'), datatype=XSD.dateTimeStamp)))


print(g.serialize(format='turtle'))
g.serialize(destination=f"/workspaces/ontologie-peuplement/data/rdf/{COMMUNE}_landmarks_initial_plots.ttl", format='turtle')

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix atype: <http://rdf.geohistoricaldata.org/id/codes/address/attributeType/> .
@prefix cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/> .
@prefix cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/> .
@prefix ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix landmark: <http://data.ign.fr/id/landmark/> .
@prefix pnature: <http://data.ign.fr/id/codes/cadastre/plotNature/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<_:N00c7f96b90414ffc9cec78c774da05b5> a add:LandmarkRelation ;
    add:locatum landmark:46e159fb-38f1-4430-a7f0-d380b7e2dfad ;
    add:relatum landmark:da6a5c2c-e86d-43bb-8950-7169bd0df60

<Graph identifier=N41df0f0ac1e841738f3e644628bafc41 (<class 'rdflib.graph.Graph'>)>